In [2]:
import numpy as np
import cv2, os, random, glob
from matplotlib import pyplot as plt
from collections import Counter


# 1 pkm card size pixel = 2078(h) x 1488(w)
# 1 cm ~ 236.2 pixels
# dpi = 600


In [3]:
batch_path = './Batch/batch_en.jpg'
x = [100, 1800, 3500, 5200]
y = [200, 2600, 5000, 7400]
h = 2300
w = 1700

for i in range(4):
    for j in range(4):
        index = len(glob.glob1("./OriginalImages","*.jpg"))
        img = cv2.imread(batch_path)
        crop_img = img[y[i]:y[i]+h, x[j]:x[j]+w] #crop
        cv2.imwrite('./OriginalImages/PKM_'+str(index).zfill(3)+'F_en.jpg', crop_img)
        


In [9]:
# Import required packages
import cv2
import pytesseract

# Mention the installed location of Tesseract-OCR in your system
pytesseract.pytesseract.tesseract_cmd = r'/opt/homebrew/Cellar/tesseract/5.0.0/bin/tesseract'

# Read image from which text needs to be extracted
img = cv2.imread(batch_path)

# Preprocessing the image starts

# Convert the image to gray scale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Performing OTSU threshold
ret, thresh1 = cv2.threshold(gray, 0, 255, cv2.THRESH_OTSU | cv2.THRESH_BINARY_INV)

# Specify structure shape and kernel size.
# Kernel size increases or decreases the area
# of the rectangle to be detected.
# A smaller value like (10, 10) will detect
# each word instead of a sentence.
rect_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (18, 18))

# Applying dilation on the threshold image
dilation = cv2.dilate(thresh1, rect_kernel, iterations = 1)

# Finding contours
contours, hierarchy = cv2.findContours(dilation, cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)

# Creating a copy of image
im2 = img.copy()

# A text file is created and flushed
file = open("recognized.txt", "w+")
file.write("")
file.close()

# Looping through the identified contours
# Then rectangular part is cropped and passed on
# to pytesseract for extracting text from it
# Extracted text is then written into the text file
for cnt in contours:
    x, y, w, h = cv2.boundingRect(cnt)

    # Drawing a rectangle on copied image
    rect = cv2.rectangle(im2, (x, y), (x + w, y + h), (0, 255, 0), 2)

    # Cropping the text block for giving input to OCR
    cropped = im2[y:y + h, x:x + w]

    # Open the file in append mode
    file = open("recognized.txt", "a")

    # Apply OCR on the cropped image
    text = pytesseract.image_to_string(cropped)

    # Appending the text into file
    file.write(text)
    file.write("\n")

    # Close the file
    file.close


In [34]:
from pytesseract import *
from PIL import Image

image = cv2.imread("./CroppedImages/PKM_037F_en.jpg")
text = image_to_string(image)
text


'PlusPower ©\n\nAttach PlusPower to | of your Pokémon.\nDiscard this card at the end of your turn.\n\nIf the Pokémon PlusPower is attached to\nattacks, the attack does 10 more damage\nto the Active Pokémon (before applying\nWeakness and Resistance).\n\n \n'